In [ ]:
import heapq
import math

class MSTApproxSolver:
    def __init__(self, filename):

        distance_matrix = []
        euc_coordinates = []
        explicit = True

        with open(filename, 'r') as f:
            lines = f.readlines()
            read_distances = False  

            for line in lines: 
                line = line.strip()

                if line.startswith('EOF'):
                    break
                elif line.startswith('EDGE_WEIGHT_TYPE'):
                    if 'EXPLICIT' in line:
                        explicit = True
                    else:
                        explicit = False
                elif line.startswith('EDGE_WEIGHT_SECTION') or line.startswith('NODE_COORD_SECTION'):
                    read_distances = True
                elif explicit and read_distances:
                    elements = line.split()
                    distance_matrix.append([int(e) for e in elements])
                elif explicit == False and read_distances: 
                    elements = line.split()
                    x = int(elements[1])
                    y = int(elements[2])
                    euc_coordinates.append((x, y))

        if explicit:
            self.distance_matrix = distance_matrix
            self.n = len(distance_matrix)
            self.explicit = True
        else:
            self.coordinates = euc_coordinates
            self.n = len(euc_coordinates)
            self.explicit = False

    def get_cost(self, i, j):
        if self.explicit:
            return self.distance_matrix[i][j]    
        else:
            x1, y1 = self.coordinates[i]
            x2, y2 = self.coordinates[j]
            return math.ceil(((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5)

    def prim_mst(self):

        self.mst_edges = []  # Stores (u, v, weight) edges in MST
        total_weight = 0
        visited = [False] * self.n
        min_heap = [(0, 0, -1)]  # (weight, vertex, parent)

        while len(self.mst_edges) < self.n - 1:
            weight, u, parent = heapq.heappop(min_heap)

            if visited[u]:
                continue
            
            visited[u] = True
            if parent != -1:
                self.mst_edges.append((parent, u, weight))
                total_weight += weight

            for v in range(self.n):
                if not visited[v] and self.get_cost(u, v) > 0:
                    heapq.heappush(min_heap, (self.get_cost(u, v), v, u))

    def dfs_mst(self, start): 

        visited = [False] * self.n
        self.tour = []
        stack = [start]
    
        while stack:
            u = stack.pop()  # Pop a vertex from the stack
            if visited[u]:
                continue  # Skip already visited vertices
            
            visited[u] = True
            self.tour.append(u)  # Add the vertex to the tour

            # Add all unvisited neighbors to the stack
            for edge in self.mst_edges:
                # u -> v or v -> u (undirected graph)
                if edge[0] == u and not visited[edge[1]]:
                    stack.append(edge[1])
                elif edge[1] == u and not visited[edge[0]]:
                    stack.append(edge[0])
    
        return self.tour  # Return the collected tour


    def two_approx(self, start):

        self.prim_mst()

        self.dfs_mst(start)

        unique_tour = []
        cost = 0
        seen = set()

    
        for i, vertex in enumerate(self.tour):
            if vertex not in seen:
                unique_tour.append(vertex)
                seen.add(vertex)

                # If not the first vertex, calculate the cost from the last added vertex
                if len(unique_tour) > 1:
                    prev_vertex = unique_tour[-2]
                    cost += self.get_cost(prev_vertex, vertex)

        # To return to the start vertex (completing the cycle)
        cost += self.get_cost(unique_tour[-1], unique_tour[0])

        return cost, unique_tour

In [ ]:
# filename = "../tsplib_converted_instances/01_small/gr17_converted.tsp"
filename = "../tsplib_instances_new/01_small/kroA100.tsp"

# Solve using 2-Opt 
two_approx_solver = MSTApproxSolver(filename)
ta_cost, ta_path = two_approx_solver.two_approx(0)

print("\nTwo-Approximation-Solver - Minimum Cost:", ta_path)
print("\nPath: ", ta_cost)


Two-Approximation-Solver - Minimum Cost: [0, 42, 40, 41, 63, 72, 32, 62, 22, 55, 15, 48, 23, 61, 21, 27, 73, 29, 47, 46, 35, 68, 20, 60, 28, 4, 14, 56, 36, 19, 69, 59, 70, 44, 26, 12, 53, 51, 33, 66, 75, 74, 3, 67, 5, 50, 16, 39, 11, 25, 45, 7, 18, 34, 52, 10, 65, 64, 37, 9, 30, 57, 71, 38, 8, 31, 49, 24, 54, 17, 43, 2, 13, 58, 6, 1]

Path:  769
